In [1]:
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import torch
import torch.nn as nn
import torch.optim as optim

import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

# 定义一个简单的CNN模型
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        # Layer 1: Conv2D
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(3, 3), padding=1)  # 输入通道 1，输出通道 32
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2))  # 空间尺寸减半
        
        # Layer 2: Conv2D
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3, 3), padding=1)  # 输出通道 64
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))
        
        # Layer 3: Conv2D
        self.conv3 = nn.Conv2d(64, 128, kernel_size=(3, 3), padding=1)  # 输出通道 128
        self.batchnorm3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=(2, 2))
        
        # Flatten Layer
        self.flatten = nn.Flatten()
        
        # Fully Connected Layer
        self.fc1 = nn.Linear(128 * 8 * 125, 256)  # 根据池化后特征图的尺寸调整
        self.fc2 = nn.Linear(256, 1)  # 二分类问题
        
    def forward(self, x):
        # Layer 1
        x = F.relu(self.conv1(x))
        x = self.batchnorm1(x)
        x = self.pool1(x)
        
        # Layer 2
        x = F.relu(self.conv2(x))
        x = self.batchnorm2(x)
        x = self.pool2(x)
        
        # Layer 3
        x = F.relu(self.conv3(x))
        x = self.batchnorm3(x)
        x = self.pool3(x)
        
        # Flatten
        x = self.flatten(x)
        
        # Fully Connected Layers
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))  # 使用 Sigmoid 激活函数
        
        return x


In [2]:
import numpy as np
import torch


# 加载模型
model = SimpleCNN()
model.load_state_dict(torch.load('./model.pth'))
model.eval()

# 加载数据
data = np.load('data.npy')  # 假设原始数据形状为 (num_trials, channels, time_steps)
label = np.load('label.npy')
# 数据预处理
data = data.transpose(0, 2, 1)  # 调整轴顺序为 (num_trials, time_steps, channels)
data = data[:, np.newaxis, :, :]  # 增加通道维度，变为 (num_trials, 1, time_steps, channels)
data_tensor = torch.from_numpy(data).float()

# 输出预测结果
with torch.no_grad():
    for i in range(data_tensor.size(0)):  # 遍历每个样本
        input_data = data_tensor[i:i+1]  # 取一个样本，保持 batch 维度
        output = model(input_data).item()  # 获取预测值
        print(f"Sample {i}: Prediction={output} {label[i]}")
        



Sample 0: Prediction=0.000531426107045263 [0]
Sample 1: Prediction=0.15104994177818298 [1]
Sample 2: Prediction=0.5029907822608948 [1]
Sample 3: Prediction=0.03834341838955879 [0]
Sample 4: Prediction=0.9734413027763367 [0]
Sample 5: Prediction=0.996880054473877 [1]
Sample 6: Prediction=0.00071568792918697 [0]
Sample 7: Prediction=0.9988779425621033 [1]
Sample 8: Prediction=0.9962947964668274 [1]
Sample 9: Prediction=0.0011126544559374452 [0]
Sample 10: Prediction=0.0014088000170886517 [0]
Sample 11: Prediction=0.008728448301553726 [1]
Sample 12: Prediction=0.9999145269393921 [1]
Sample 13: Prediction=0.0001712232769932598 [0]
Sample 14: Prediction=6.198680057423189e-05 [0]
Sample 15: Prediction=0.999944806098938 [1]
Sample 16: Prediction=0.0006858262931928039 [0]
Sample 17: Prediction=0.9975202679634094 [1]
Sample 18: Prediction=0.9999736547470093 [1]
Sample 19: Prediction=0.0006776926456950605 [0]
Sample 20: Prediction=0.0008668156806379557 [0]
Sample 21: Prediction=0.999881744384765